# Generative Adversarial Networks (GANs)

TensorFlow and Keras provide a robust platform for implementing GANs. The following code demonstrates a basic GAN for generating synthetic images of MNIST digits.


In [1]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Collecting tensorflow



You should consider upgrading via the 'c:\Users\sirpr\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sirpr\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


TypeError: unhashable type: 'list'